In this notebook we'll try to figure out if we can use nibabel to create and save a cifiti image, which can be visuzlized using the connectome workbench.
For example if you use the Glasser or Cole Anticivec parcellation, they are in cifti format. It would be good if you can dump values onto each parcel/ROI and visualize them.


In [1]:
import nibabel as nib # we use version
print("nibabel version:" + nib.__version__)
import numpy as np
import pandas as pd

nibabel version:3.1.0


Let us start by loading an example CIFTI file. We will use cifti files from the Cole Anticivec parcellation repository: 
https://github.com/ColeLab/ColeAnticevicNetPartition \
In this repo there are also workbench "scene" files you can use to visualize stuff in the connectome workbench

We will play around with two different types of cifti files \
dscalar, which dumps a scalar value to each surface vertex and each volume voxle (hence "dense") \
dlabel, which dumps a label integer value to each vertex/voxel, this is the "parcel ROI file" \

These files will be loaded as cifti2 object with nibabel

In [2]:
template_cifti_dlabelfile = nib.load("../ColeAnticevicNetPartition/CortexSubcortex_ColeAnticevic_NetPartition_wSubcorGSR_parcels_LR.dlabel.nii")
template_cifti_dscalarfile = nib.load("../ColeAnticevicNetPartition/CortexSubcortex_ColeAnticevic_NetPartition_wSubcorGSR_parcels_LR_ReorderedByNetworks.dscalar.nii")


we can load the data assoicated with each object and exaime its content

In [3]:
template_cifti_dscalarfile.get_fdata().shape

(1, 91282)

In [4]:
len(np.unique(template_cifti_dlabelfile.get_fdata()))

718

So there are 718 unique ROIs in the dlabel file from the Cole Anticvic parcellation. How do we know what each integer represents? They provided a text file to label those.

In [5]:
label_df = pd.read_csv("../ColeAnticevicNetPartition/CortexSubcortex_ColeAnticevic_NetPartition_wSubcorGSR_parcels_LR_LabelKey.txt", sep = "\t")

In [6]:
label_df.head()

,INDEX,KEYVALUE,LABEL,RED,GREEN,BLUE,ALPHA,HEMISPHERE,NETWORK,NETWORKKEY,NETWORKSORTEDORDER
0,1,1,Visual1-04_L-Ctx,0,0,255,255,L,Visual1,1,1
1,2,2,Visual2-28_L-Ctx,100,0,255,255,L,Visual2,2,70
2,3,3,Visual2-29_L-Ctx,100,0,255,255,L,Visual2,2,71
3,4,4,Visual2-30_L-Ctx,100,0,255,255,L,Visual2,2,72
4,5,5,Visual2-31_L-Ctx,100,0,255,255,L,Visual2,2,73


In [7]:
#Here are all the thalamus parcels
label_df.loc[label_df['LABEL'].str.contains('Thalamus')]
print(len(label_df.loc[label_df['LABEL'].str.contains('Thalamus')]))

38


now let us write a fake value, say -1000, to all the thalamus voxels to a new dscalar file, using the thalamus parcel labels from the dlabel cifti file to index their locations. 

In [8]:
#need to reload to link to new data object
template_cifti_dlabelfile = nib.load("../ColeAnticevicNetPartition/CortexSubcortex_ColeAnticevic_NetPartition_wSubcorGSR_parcels_LR.dlabel.nii")
new_label_data = template_cifti_dlabelfile.get_fdata()

In [9]:
for key in label_df.loc[label_df['LABEL'].str.contains('Thalamus')]['KEYVALUE']:
    print(key)
    new_label_data[0,new_label_data[0,:]==key]=-1000

1550
1560
1570
1580
1590
1600
1610
1620
1630
2280
2290
3270
3280
4360
4370
4380
4390
5230
5240
7470
7480
8230
8240
8250
8260
8270
8280
8290
8300
8310
9270
9280
9290
9300
9310
9320
10280
10290


now we will create a new cifti dscalr file with this new_label_data, by using the header info from a cifti dscalar file as a template, and replace the data object with new_label_data \
https://nipy.org/nibabel/reference/nibabel.cifti2.html#nibabel.cifti2.cifti2.Cifti2Image


In [10]:
new_cifti = nib.cifti2.cifti2.Cifti2Image(new_label_data, template_cifti_dscalarfile.header)
nib.save(new_cifti, 'new_cifti.dscalar.nii')

In workbench the thalamus is in dark blue

![](test1.png)

Now let us replace each thalamus parcel with a unique value. We will do this in the order of the parcels listed in Cole/Anticivec's ROI list. Imagin that you have a vector of data that are correctly ordered (say from the thalamocortical FC matrix), you can use this logic to display custom values in the correct anatomical location. The key is you have to make sure the order of your custom thalamic data matches the parcel order in the Cole Anticivec ROI parcel file.

In [11]:
template_cifti_dlabelfile = nib.load("../ColeAnticevicNetPartition/CortexSubcortex_ColeAnticevic_NetPartition_wSubcorGSR_parcels_LR.dlabel.nii")
label_data = template_cifti_dlabelfile.get_fdata()
new_label_data = label_data.copy() # create copy so not to link to original copy
new_label_data[new_label_data!=0] = 0

i = 1
for key in label_df.loc[label_df['LABEL'].str.contains('Thalamus')]['KEYVALUE']:
    new_label_data[0,label_data[0,:]==key]=i
    i = i+1
    

In [12]:
np.unique(new_label_data)

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38.])

In [13]:
#save and visualize
new_cifti = nib.cifti2.cifti2.Cifti2Image(new_label_data, template_cifti_dscalarfile.header)
nib.save(new_cifti, 'new_cifti.dscalar.nii')

In workbench each thalamus parcel has a unique color. And notice that everything else got set to 0.

![](test2.png)

So in the Cole/Anticivec parcellation there are 38 unique parcels in the thalamus. Here we relabeled them using the original ascending order in the dlabel cifti file, and visualize those integer values. As long as you have a vector of 38 thalamic parcel values that you are sure matches the  order in the original dlabel parcellation file, you can then write custom cifti files using the logic described above to visualize those values.

Now, let us try to visualize some of our own thalamus data from the thalamocortical FC project.

In [37]:
#load HCP matrices
rest = np.load('data/ColeAntecivic_wGSR_allrfMRI_ThaAr_X_CtxAr.npy')
wm = np.load('data/ColeAntecivic_wGSR_allWM_ThaAr_X_CtxAr.npy')
lan = np.load('data/ColeAntecivic_wGSR_allLANGUAGE_ThaAr_X_CtxAr.npy')
motor = np.load('data/ColeAntecivic_wGSR_allMOTOR_ThaAr_X_CtxAr.npy')
social = np.load('data/ColeAntecivic_wGSR_allSOCIAL_ThaAr_X_CtxAr.npy')
gamb = np.load('data/ColeAntecivic_wGSR_allGAMBLING_ThaAr_X_CtxAr.npy')
task = np.load('data/ColeAntecivic_wGSR_alltask_ThaAr_X_CtxAr.npy')

In [25]:
rest_v_task = task-rest
rest_v_task = np.mean(rest_v_task, axis=1)

In [30]:
def write_tha_cifti(input_vector, fn):
    template_cifti_dlabelfile = nib.load("../ColeAnticevicNetPartition/CortexSubcortex_ColeAnticevic_NetPartition_wSubcorGSR_parcels_LR.dlabel.nii")
    label_data = template_cifti_dlabelfile.get_fdata()
    new_label_data = label_data.copy() # create copy so not to link to original copy
    new_label_data[new_label_data!=0] = 0


    for i, key in enumerate(label_df.loc[label_df['LABEL'].str.contains('Thalamus')]['KEYVALUE']):
        new_label_data[0,label_data[0,:]==key]=input_vector[i]
    
    new_cifti = nib.cifti2.cifti2.Cifti2Image(new_label_data, template_cifti_dscalarfile.header)
    nib.save(new_cifti, fn)
    

In [31]:
write_tha_cifti(rest_v_task, 'rest_v_task.dscalar.nii')

In [34]:
rest_v_wm = wm-rest
rest_v_wm = np.mean(rest_v_wm, axis=1)
write_tha_cifti(rest_v_wm, 'rest_v_wm.dscalar.nii')

In [35]:
rest_v_lan = lan-rest
rest_v_lan = np.mean(rest_v_lan, axis=1)
write_tha_cifti(rest_v_lan, 'rest_v_lan.dscalar.nii')

In [42]:
rest_v_gamb = gamb-rest
rest_v_gamb = np.mean(rest_v_gamb, axis=1)
write_tha_cifti(rest_v_gamb, 'rest_v_gamb.dscalar.nii')

In [43]:
rest_v_social = social-rest
rest_v_social = np.mean(rest_v_social, axis=1)
write_tha_cifti(rest_v_social, 'rest_v_social.dscalar.nii')

In [44]:
rest_v_motor = motor-rest
rest_v_motor = np.mean(rest_v_motor, axis=1)
write_tha_cifti(rest_v_motor, 'rest_v_motor.dscalar.nii')